In [5]:
import pandas as pd
import requests
import geopandas as gpd
from shapely.geometry import Point, shape
import zipfile
import os
from geopy.geocoders import Nominatim

# Initialize geocoder
geolocator = Nominatim(user_agent="geocoding-script")

def assemble_address(row):
    address_parts = [str(row['Address Line 1'])]
    if pd.notna(row['Address Line 2']):
        address_parts.append(str(row['Address Line 2']))
    address_parts.append(f"{str(row['City'])}, {str(row['State'])} {str(row['Zip Code'])}")
    if pd.notna(row['Zip Plus 4']):
        address_parts.append(str(row['Zip Plus 4']))
    return " ".join(address_parts)

def geocode_address(address):
    location = geolocator.geocode(address, addressdetails=True, limit=1)
    if location:
        return {
            "address": address,
            "lat": location.latitude,
            "lon": location.longitude
        }
    else:
        return None

def get_building_footprints(lat, lon):
    overpass_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json];
    (
      way["building"](around:20, {lat}, {lon});
      relation["building"](around:20, {lat}, {lon});
    );
    out body;
    out skel qt;
    """
    response = requests.get(overpass_url, params={'data': query})
    data = response.json()
    print("Overpass API Response:")
    print(data)  # Print the entire response for inspection
    return data

def extract_building_polygons(footprints_data):
    nodes_dict = {node['id']: (node['lon'], node['lat']) for node in footprints_data['elements'] if node['type'] == 'node'}
    
    print(f"Total nodes: {len(nodes_dict)}")
    
    polygons = []
    
    for element in footprints_data['elements']:
        if element['type'] == 'way' and 'tags' in element and 'building' in element['tags']:
            coordinates = []
            missing_nodes = []
            for node_id in element['nodes']:
                if node_id in nodes_dict:
                    coordinates.append(nodes_dict[node_id])
                else:
                    missing_nodes.append(node_id)
            
            if missing_nodes:
                print(f"Missing nodes for way {element['id']}: {missing_nodes}")
            
            if len(coordinates) >= 3:
                polygon = shape({
                    "type": "Polygon",
                    "coordinates": [coordinates]
                })
                polygons.append(polygon)
    
    return polygons

def read_addresses_from_excel(file_path):
    df = pd.read_excel(file_path)
    df['Full Address'] = df.apply(assemble_address, axis=1)
    return df['Full Address'].tolist()

def process_addresses(file_path):
    addresses = read_addresses_from_excel(file_path)
    results = []

    for address in addresses:
        print(f"Processing address: {address}")
        
        address_data = geocode_address(address)
        if address_data:
            lat = address_data['lat']
            lon = address_data['lon']
            point = Point(float(lon), float(lat))
            
            # Get building footprints
            footprints_data = get_building_footprints(lat, lon)
            polygons = extract_building_polygons(footprints_data)
            
            for polygon in polygons:
                result = {
                    "Address": address_data['address'],
                    "Latitude": lat,
                    "Longitude": lon,
                    "geometry": polygon
                }
                results.append(result)
            
            if not polygons:
                result = {
                    "Address": address_data['address'],
                    "Latitude": lat,
                    "Longitude": lon,
                    "geometry": point
                }
                results.append(result)
        else:
            results.append({
                "Address": address,
                "Latitude": None,
                "Longitude": None,
                "geometry": None
            })

    gdf = gpd.GeoDataFrame(results, geometry='geometry')
    
    shapefile_path = 'geocoded_addresses.shp'
    gdf.to_file(shapefile_path, driver='ESRI Shapefile')

    with zipfile.ZipFile('geocoded_addresses.zip', 'w') as z:
        for root, dirs, files in os.walk('.'):
            for file in files:
                if file.startswith('geocoded_addresses') and file.endswith(('.shp', '.shx', '.dbf', '.prj')):
                    z.write(os.path.join(root, file), file)

    for file in os.listdir('.'):
        if file.startswith('geocoded_addresses') and file.endswith(('.shp', '.shx', '.dbf', '.prj')):
            os.remove(file)

if __name__ == "__main__":
    excel_file_path = r"data/data/TWDB_Active_Policies_Subset.xlsx"  # Update this path to your Excel file
    process_addresses(excel_file_path)

Processing address: 1001 ELMWOOD DR ABILENE, TX 79605
Overpass API Response:
{'version': 0.6, 'generator': 'Overpass API 0.7.62.1 084b4234', 'osm3s': {'timestamp_osm_base': '2024-08-19T17:09:00Z', 'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'}, 'elements': [{'type': 'way', 'id': 1099454022, 'nodes': [10065493298, 10065493299, 10065493300, 10065493301, 10065493302, 10065493303, 10065493304, 10065493305, 10065493306, 10065493307, 10065493308, 10065493309, 10065493310, 10065493311, 10065493312, 10065493313, 10065493314, 10065493315, 10065493298], 'tags': {'addr:state': 'TX', 'building': 'yes', 'source': 'esri/Abilene_TX_Buildings'}}, {'type': 'way', 'id': 1099454022, 'nodes': [10065493298, 10065493299, 10065493300, 10065493301, 10065493302, 10065493303, 10065493304, 10065493305, 10065493306, 10065493307, 10065493308, 10065493309, 10065493310, 10065493311, 10065493312, 10065493313, 10065493314, 10065493315, 100654932

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=1901+BRIARWOOD+ST+ABILENE%2C+TX+79603+3314.0&format=json&limit=1&addressdetails=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [6]:
import pandas as pd
import requests
import geopandas as gpd
from shapely.geometry import Point, shape
import zipfile
import os
from geopy.geocoders import Nominatim

# Initialize geocoder
geolocator = Nominatim(user_agent="geocoding-script")

def assemble_address(row):
    address_parts = [str(row['Address Line 1'])]
    if pd.notna(row['Address Line 2']):
        address_parts.append(str(row['Address Line 2']))
    address_parts.append(f"{str(row['City'])}, {str(row['State'])} {str(row['Zip Code'])}")
    if pd.notna(row['Zip Plus 4']):
        address_parts.append(str(row['Zip Plus 4']))
    return " ".join(address_parts)

def geocode_address(address, retries=3, delay=5):
    attempt = 0
    while attempt < retries:
        try:
            location = geolocator.geocode(address, addressdetails=True, limit=1, timeout=10)  # Increased timeout
            if location:
                return {
                    "address": address,
                    "lat": location.latitude,
                    "lon": location.longitude
                }
            else:
                return None
        except Exception as e:
            print(f"Error geocoding address '{address}': {e}")
            attempt += 1
            time.sleep(delay)  # Wait before retrying
    return None

def get_building_footprints(lat, lon):
    overpass_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json];
    (
      way["building"](around:20, {lat}, {lon});
      relation["building"](around:20, {lat}, {lon});
    );
    out body;
    out skel qt;
    """
    response = requests.get(overpass_url, params={'data': query})
    data = response.json()
    print("Overpass API Response:")
    print(data)  # Print the entire response for inspection
    return data

def extract_building_polygons(footprints_data):
    nodes_dict = {node['id']: (node['lon'], node['lat']) for node in footprints_data['elements'] if node['type'] == 'node'}
    
    print(f"Total nodes: {len(nodes_dict)}")
    
    polygons = []
    
    for element in footprints_data['elements']:
        if element['type'] == 'way' and 'tags' in element and 'building' in element['tags']:
            coordinates = []
            missing_nodes = []
            for node_id in element['nodes']:
                if node_id in nodes_dict:
                    coordinates.append(nodes_dict[node_id])
                else:
                    missing_nodes.append(node_id)
            
            if missing_nodes:
                print(f"Missing nodes for way {element['id']}: {missing_nodes}")
            
            if len(coordinates) >= 3:
                polygon = shape({
                    "type": "Polygon",
                    "coordinates": [coordinates]
                })
                polygons.append(polygon)
    
    return polygons

def read_addresses_from_excel(file_path):
    df = pd.read_excel(file_path)
    df['Full Address'] = df.apply(assemble_address, axis=1)
    return df['Full Address'].tolist()

def process_addresses(file_path):
    addresses = read_addresses_from_excel(file_path)
    results = []

    for address in addresses:
        print(f"Processing address: {address}")
        
        address_data = geocode_address(address)
        if address_data:
            lat = address_data['lat']
            lon = address_data['lon']
            point = Point(float(lon), float(lat))
            
            # Get building footprints
            footprints_data = get_building_footprints(lat, lon)
            polygons = extract_building_polygons(footprints_data)
            
            for polygon in polygons:
                result = {
                    "Address": address_data['address'],
                    "Latitude": lat,
                    "Longitude": lon,
                    "geometry": polygon
                }
                results.append(result)
            
            if not polygons:
                result = {
                    "Address": address_data['address'],
                    "Latitude": lat,
                    "Longitude": lon,
                    "geometry": point
                }
                results.append(result)
        else:
            results.append({
                "Address": address,
                "Latitude": None,
                "Longitude": None,
                "geometry": None
            })

    gdf = gpd.GeoDataFrame(results, geometry='geometry')
    
    shapefile_path = 'geocoded_addresses.shp'
    gdf.to_file(shapefile_path, driver='ESRI Shapefile')

    with zipfile.ZipFile('geocoded_addresses.zip', 'w') as z:
        for root, dirs, files in os.walk('.'):
            for file in files:
                if file.startswith('geocoded_addresses') and file.endswith(('.shp', '.shx', '.dbf', '.prj')):
                    z.write(os.path.join(root, file), file)

    for file in os.listdir('.'):
        if file.startswith('geocoded_addresses') and file.endswith(('.shp', '.shx', '.dbf', '.prj')):
            os.remove(file)

if __name__ == "__main__":
    excel_file_path = r"data/data/TWDB_Active_Policies_Subset.xlsx"  # Update this path to your Excel file
    process_addresses(excel_file_path)

Processing address: 1001 ELMWOOD DR ABILENE, TX 79605
Overpass API Response:
{'version': 0.6, 'generator': 'Overpass API 0.7.62.1 084b4234', 'osm3s': {'timestamp_osm_base': '2024-08-19T17:12:04Z', 'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'}, 'elements': [{'type': 'way', 'id': 1099454022, 'nodes': [10065493298, 10065493299, 10065493300, 10065493301, 10065493302, 10065493303, 10065493304, 10065493305, 10065493306, 10065493307, 10065493308, 10065493309, 10065493310, 10065493311, 10065493312, 10065493313, 10065493314, 10065493315, 10065493298], 'tags': {'addr:state': 'TX', 'building': 'yes', 'source': 'esri/Abilene_TX_Buildings'}}, {'type': 'way', 'id': 1099454022, 'nodes': [10065493298, 10065493299, 10065493300, 10065493301, 10065493302, 10065493303, 10065493304, 10065493305, 10065493306, 10065493307, 10065493308, 10065493309, 10065493310, 10065493311, 10065493312, 10065493313, 10065493314, 10065493315, 100654932

C:\Users\bdhungan\AppData\Local\miniconda3\Lib\site-packages\pyogrio\raw.py:698: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  warnings.warn(


In [8]:
import pandas as pd
import requests
import geopandas as gpd
from shapely.geometry import Point, shape, box
import os
import time
from geopy.geocoders import Nominatim
import leafmap
import folium

# Initialize geocoder
geolocator = Nominatim(user_agent="geocoding-script")

def assemble_address(row):
    # Simplify address assembly by excluding Address Line 2 and Zip Plus 4
    address_parts = [str(row['Address Line 1'])]
    address_parts.append(f"{str(row['City'])}, {str(row['State'])} {str(row['Zip Code'])}")
    return " ".join(address_parts)

def geocode_address(address, retries=3, delay=5):
    attempt = 0
    while attempt < retries:
        try:
            location = geolocator.geocode(address, addressdetails=True, limit=1, timeout=10)  # Increased timeout
            if location:
                return {
                    "address": address,
                    "lat": location.latitude,
                    "lon": location.longitude
                }
            else:
                return None
        except Exception as e:
            print(f"Error geocoding address '{address}': {e}")
            attempt += 1
            time.sleep(delay)  # Wait before retrying
    return None

def get_building_footprints(lat, lon):
    overpass_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json];
    (
      way["building"](around:20, {lat}, {lon});
      relation["building"](around:20, {lat}, {lon});
    );
    out body;
    out skel qt;
    """
    response = requests.get(overpass_url, params={'data': query})
    data = response.json()
    print("Overpass API Response:")
    print(data)  # Print the entire response for inspection
    return data

def extract_building_polygons(footprints_data):
    nodes_dict = {node['id']: (node['lon'], node['lat']) for node in footprints_data['elements'] if node['type'] == 'node'}
    
    print(f"Total nodes: {len(nodes_dict)}")
    
    polygons = []
    bounding_boxes = []
    
    for element in footprints_data['elements']:
        if element['type'] == 'way' and 'tags' in element and 'building' in element['tags']:
            coordinates = []
            missing_nodes = []
            for node_id in element['nodes']:
                if node_id in nodes_dict:
                    coordinates.append(nodes_dict[node_id])
                else:
                    missing_nodes.append(node_id)
            
            if missing_nodes:
                print(f"Missing nodes for way {element['id']}: {missing_nodes}")
            
            if len(coordinates) >= 3:
                polygon = shape({
                    "type": "Polygon",
                    "coordinates": [coordinates]
                })
                polygons.append(polygon)
                
                # Calculate bounding box
                lons, lats = zip(*coordinates)
                min_lon, max_lon = min(lons), max(lons)
                min_lat, max_lat = min(lats), max(lats)
                bbox = box(min_lon, min_lat, max_lon, max_lat)
                bounding_boxes.append(bbox)
    
    return polygons, bounding_boxes

def read_addresses_from_excel(file_path):
    df = pd.read_excel(file_path)
    df['Full Address'] = df.apply(assemble_address, axis=1)
    return df

def process_addresses(file_path):
    df = read_addresses_from_excel(file_path)
    results = []

    for _, row in df.iterrows():
        address = row['Full Address']
        print(f"Processing address: {address}")
        
        address_data = geocode_address(address)
        if address_data:
            lat = address_data['lat']
            lon = address_data['lon']
            point = Point(float(lon), float(lat))
            
            # Get building footprints
            footprints_data = get_building_footprints(lat, lon)
            polygons, bounding_boxes = extract_building_polygons(footprints_data)
            
            for polygon, bbox in zip(polygons, bounding_boxes):
                result = {
                    "Address": address_data['address'],
                    "Latitude": lat,
                    "Longitude": lon,
                    "geometry": polygon,
                    "bounding_box": bbox
                }
                results.append(result)
            
            if not polygons:
                result = {
                    "Address": address_data['address'],
                    "Latitude": lat,
                    "Longitude": lon,
                    "geometry": point,
                    "bounding_box": None
                }
                results.append(result)
        else:
            results.append({
                "Address": address,
                "Latitude": None,
                "Longitude": None,
                "geometry": None,
                "bounding_box": None
            })

    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame(results, geometry='geometry')
    
    # Set CRS (WGS 84)
    gdf.set_crs(epsg=4326, inplace=True)
    
    return gdf

def visualize_on_map(gdf):
    m = leafmap.Map(width=800, height=500)
    
    # Plot the geocoded points
    gdf.explore(
        m=m,
        marker_type='marker',
        marker_kwds={
            'icon': folium.Icon(color='#fdbb84', icon='hurricane', prefix='fa')
        }
    )
    
    m.zoom_to_gdf(gdf)
    return m

if __name__ == "__main__":
    excel_file_path = r"data/data/TWDB_Active_Policies_Subset.xlsx"  # Update this path to your Excel file
    address_gdf = process_addresses(excel_file_path)
    map_ = visualize_on_map(address_gdf)
    map_

Processing address: 1001 ELMWOOD DR ABILENE, TX 79605
Overpass API Response:
{'version': 0.6, 'generator': 'Overpass API 0.7.62.1 084b4234', 'osm3s': {'timestamp_osm_base': '2024-08-19T17:35:41Z', 'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'}, 'elements': [{'type': 'way', 'id': 1099454022, 'nodes': [10065493298, 10065493299, 10065493300, 10065493301, 10065493302, 10065493303, 10065493304, 10065493305, 10065493306, 10065493307, 10065493308, 10065493309, 10065493310, 10065493311, 10065493312, 10065493313, 10065493314, 10065493315, 10065493298], 'tags': {'addr:state': 'TX', 'building': 'yes', 'source': 'esri/Abilene_TX_Buildings'}}, {'type': 'way', 'id': 1099454022, 'nodes': [10065493298, 10065493299, 10065493300, 10065493301, 10065493302, 10065493303, 10065493304, 10065493305, 10065493306, 10065493307, 10065493308, 10065493309, 10065493310, 10065493311, 10065493312, 10065493313, 10065493314, 10065493315, 100654932

AttributeError: 'Map' object has no attribute 'add_child'

In [15]:
import pandas as pd
import requests
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from shapely.geometry import Point, box
import geopandas as gpd

# Initialize geocoder and rate limiter
locator = Nominatim(user_agent='geocoding-script', timeout=10)
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

# Function to get bounding box
def get_bounding_box(lat, lon, radius=20):
    overpass_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json];
    (
      way["building"](around:{radius}, {lat}, {lon});
      relation["building"](around:{radius}, {lat}, {lon});
    );
    out body;
    out skel qt;
    """
    response = requests.get(overpass_url, params={'data': query})
    data = response.json()
    
    nodes_dict = {node['id']: (node['lon'], node['lat']) for node in data['elements'] if node['type'] == 'node'}
    
    lons, lats = [], []
    for element in data['elements']:
        if element['type'] == 'way':
            for node_id in element['nodes']:
                if node_id in nodes_dict:
                    lons.append(nodes_dict[node_id][0])
                    lats.append(nodes_dict[node_id][1])
    
    if lons and lats:
        min_lon, max_lon = min(lons), max(lons)
        min_lat, max_lat = min(lats), max(lats)
        return box(min_lon, min_lat, max_lon, max_lat)
    return None

# Read addresses from Excel
def read_addresses(file_path):
    df = pd.read_excel(file_path)
    df['Full_Address'] = df.apply(lambda row: f"{row['Address Line 1']}, {row['City']}, {row['State']} {row['Zip Code']}", axis=1)
    return df

# Geocode addresses and add bounding box
def geocode_addresses(df):
    from tqdm import tqdm
    tqdm.pandas()
    
    # Apply geocoding
    df['location'] = df['Full_Address'].progress_apply(geocode)
    df['latitude'] = df['location'].apply(lambda loc: loc.latitude if loc else None)
    df['longitude'] = df['location'].apply(lambda loc: loc.longitude if loc else None)
    
    # Add bounding box
    df['bounding_box'] = df.apply(lambda row: get_bounding_box(row['latitude'], row['longitude']) if row['latitude'] and row['longitude'] else None, axis=1)
    
    return df

# Convert DataFrame to GeoDataFrame and save as shapefile
def save_as_shapefiles(df, valid_shapefile_path, invalid_shapefile_path):
    # Filter out rows with NaN locations
    valid_df = df.dropna(subset=['latitude', 'longitude'])
    invalid_df = df[df['latitude'].isna() | df['longitude'].isna()]

    # Convert valid DataFrame to GeoDataFrame
    valid_df['geometry'] = valid_df.apply(lambda row: Point(row['longitude'], row['latitude']) if row['latitude'] and row['longitude'] else None, axis=1)
    gdf_valid = gpd.GeoDataFrame(valid_df, geometry='geometry', crs='EPSG:4326')
    
    # Save to shapefile
    gdf_valid.to_file(valid_shapefile_path, driver='ESRI Shapefile')

    # Convert invalid DataFrame to GeoDataFrame
    invalid_df['geometry'] = None
    gdf_invalid = gpd.GeoDataFrame(invalid_df, geometry='geometry', crs='EPSG:4326')
    
    # Save to shapefile
    gdf_invalid.to_file(invalid_shapefile_path, driver='ESRI Shapefile')

# Main execution
if __name__ == "__main__":
    excel_file_path = r"data/data/TWDB_Active_Policies_Subset.xlsx"  # Update this path to your Excel file
    valid_shapefile_path = 'geocoded_addresses_valid.shp'  # Path to save the valid shapefile
    invalid_shapefile_path = 'geocoded_addresses_invalid.shp'  # Path to save the invalid shapefile
    
    # Process addresses
    address_df = read_addresses(excel_file_path)
    address_df = geocode_addresses(address_df)
    
    # Save to shapefiles
    save_as_shapefiles(address_df, valid_shapefile_path, invalid_shapefile_path)
    
    # Print DataFrames
    print("Valid addresses DataFrame:")
    print(address_df[['Full_Address', 'latitude', 'longitude', 'bounding_box']].dropna())
    
    print("Invalid addresses DataFrame:")
    print(address_df[['Full_Address', 'latitude', 'longitude', 'bounding_box']].loc[address_df['latitude'].isna() | address_df['longitude'].isna()])

100%|██████████████████████████████████████████████████████████████████████████████████| 99/99 [01:38<00:00,  1.00it/s]


Valid addresses DataFrame:
Empty DataFrame
Columns: [Full_Address, latitude, longitude, bounding_box]
Index: []
Invalid addresses DataFrame:
                                  Full_Address  latitude  longitude  \
30        1726 SANDPIPER ST, ABILENE, TX 79602       NaN        NaN   
37     1917 SOUTHRIDGE XING, ABILENE, TX 79606       NaN        NaN   
59  2410 BUTTON WILLOW PKWY, ABILENE, TX 79606       NaN        NaN   
73  2809 BUTTON WILLOW PKWY, ABILENE, TX 79606       NaN        NaN   

   bounding_box  
30         None  
37         None  
59         None  
73         None  


In [18]:
#find out the failed addresses
failed = address_df[address_df['location'].isna()]
failed

,Community Name (Number),Address Line 1,Address Line 2,City,State,Zip Code,Zip Plus 4,Full_Address,location,latitude,longitude,bounding_box
30,"ABILENE, CITY OF (485450)",1726 SANDPIPER ST,NaN,ABILENE,TX,79602,5208.0,"1726 SANDPIPER ST, ABILENE, TX 79602",None,NaN,NaN,None
37,"ABILENE, CITY OF (485450)",1917 SOUTHRIDGE XING,NaN,ABILENE,TX,79606,1198.0,"1917 SOUTHRIDGE XING, ABILENE, TX 79606",None,NaN,NaN,None
59,"ABILENE, CITY OF (485450)",2410 BUTTON WILLOW PKWY,NaN,ABILENE,TX,79606,3510.0,"2410 BUTTON WILLOW PKWY, ABILENE, TX 79606",None,NaN,NaN,None
73,"ABILENE, CITY OF (485450)",2809 BUTTON WILLOW PKWY,NaN,ABILENE,TX,79606,3517.0,"2809 BUTTON WILLOW PKWY, ABILENE, TX 79606",None,NaN,NaN,None


In [19]:
#add latitude and longitude to the dataframe
address_df['latitude'] = address_df['location'].apply(lambda loc: loc.latitude if loc else None)
address_df['longitude'] = address_df['location'].apply(lambda loc: loc.longitude if loc else None)
address_df

,Community Name (Number),Address Line 1,Address Line 2,City,State,Zip Code,Zip Plus 4,Full_Address,location,latitude,longitude,bounding_box
0,"ABILENE, CITY OF (485450)",1001 ELMWOOD DR,NaN,ABILENE,TX,79605,NaN,"1001 ELMWOOD DR, ABILENE, TX 79605","(1001, Elmwood Drive, Abilene, Taylor County, ...",32.438368,-99.768215,None
1,"ABILENE, CITY OF (485450)",1001 MATADOR ST,NaN,ABILENE,TX,79605,3019.0,"1001 MATADOR ST, ABILENE, TX 79605","(1001, Matador Street, Abilene, Taylor County,...",32.438396,-99.756995,None
2,"ABILENE, CITY OF (485450)",10257 APACHE LN,NaN,ABILENE,TX,79601,8286.0,"10257 APACHE LN, ABILENE, TX 79601","(10257, Apache Lane, Abilene, Jones County, Te...",32.596423,-99.698836,None
3,"ABILENE, CITY OF (485450)",1034 WESTVIEW DR,NaN,ABILENE,TX,79603,4722.0,"1034 WESTVIEW DR, ABILENE, TX 79603","(1034, Westview Drive, Abilene, Taylor County,...",32.461541,-99.761730,None
4,"ABILENE, CITY OF (485450)",109 TIQUEWOOD CIR,NaN,ABILENE,TX,79605,4937.0,"109 TIQUEWOOD CIR, ABILENE, TX 79605","(109, Tiquewood Circle, Abilene, Taylor County...",32.429448,-99.768803,None
...,...,...,...,...,...,...,...,...,...,...,...,...
94,"ABILENE, CITY OF (485450)",325 MOORE DR,NaN,ABILENE,TX,79605,1619.0,"325 MOORE DR, ABILENE, TX 79605","(325, Moore Drive, Abilene, Taylor County, Tex...",32.447373,-99.771750,None
95,"ABILENE, CITY OF (485450)",3266 COLLEGE ST,NaN,ABILENE,TX,79605,4041.0,"3266 COLLEGE ST, ABILENE, TX 79605","(3266, College Street, Abilene, Taylor County,...",32.433266,-99.763239,None
96,"ABILENE, CITY OF (485450)",3301 GREEN ACRES RD,NaN,ABILENE,TX,79605,5030.0,"3301 GREEN ACRES RD, ABILENE, TX 79605","(3301, Green Acres Road, Abilene, Taylor Count...",32.429782,-99.763491,None
97,"ABILENE, CITY OF (485450)",3309 PARKCREST DR,NaN,ABILENE,TX,79605,5024.0,"3309 PARKCREST DR, ABILENE, TX 79605","(3309, Parkcrest Drive, Abilene, Taylor County...",32.427916,-99.763816,None


In [20]:
#add the geometry column with in the suitable reference system
geometry = gpd.points_from_xy(address_df.longitude, address_df.latitude)
address_gdf = gpd.GeoDataFrame(address_df, crs='EPSG:4326', geometry=geometry)
address_gdf

,Community Name (Number),Address Line 1,Address Line 2,City,State,Zip Code,Zip Plus 4,Full_Address,location,latitude,longitude,bounding_box,geometry
0,"ABILENE, CITY OF (485450)",1001 ELMWOOD DR,NaN,ABILENE,TX,79605,NaN,"1001 ELMWOOD DR, ABILENE, TX 79605","(1001, Elmwood Drive, Abilene, Taylor County, ...",32.438368,-99.768215,None,POINT (-99.76822 32.43837)
1,"ABILENE, CITY OF (485450)",1001 MATADOR ST,NaN,ABILENE,TX,79605,3019.0,"1001 MATADOR ST, ABILENE, TX 79605","(1001, Matador Street, Abilene, Taylor County,...",32.438396,-99.756995,None,POINT (-99.757 32.4384)
2,"ABILENE, CITY OF (485450)",10257 APACHE LN,NaN,ABILENE,TX,79601,8286.0,"10257 APACHE LN, ABILENE, TX 79601","(10257, Apache Lane, Abilene, Jones County, Te...",32.596423,-99.698836,None,POINT (-99.69884 32.59642)
3,"ABILENE, CITY OF (485450)",1034 WESTVIEW DR,NaN,ABILENE,TX,79603,4722.0,"1034 WESTVIEW DR, ABILENE, TX 79603","(1034, Westview Drive, Abilene, Taylor County,...",32.461541,-99.761730,None,POINT (-99.76173 32.46154)
4,"ABILENE, CITY OF (485450)",109 TIQUEWOOD CIR,NaN,ABILENE,TX,79605,4937.0,"109 TIQUEWOOD CIR, ABILENE, TX 79605","(109, Tiquewood Circle, Abilene, Taylor County...",32.429448,-99.768803,None,POINT (-99.7688 32.42945)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,"ABILENE, CITY OF (485450)",325 MOORE DR,NaN,ABILENE,TX,79605,1619.0,"325 MOORE DR, ABILENE, TX 79605","(325, Moore Drive, Abilene, Taylor County, Tex...",32.447373,-99.771750,None,POINT (-99.77175 32.44737)
95,"ABILENE, CITY OF (485450)",3266 COLLEGE ST,NaN,ABILENE,TX,79605,4041.0,"3266 COLLEGE ST, ABILENE, TX 79605","(3266, College Street, Abilene, Taylor County,...",32.433266,-99.763239,None,POINT (-99.76324 32.43327)
96,"ABILENE, CITY OF (485450)",3301 GREEN ACRES RD,NaN,ABILENE,TX,79605,5030.0,"3301 GREEN ACRES RD, ABILENE, TX 79605","(3301, Green Acres Road, Abilene, Taylor Count...",32.429782,-99.763491,None,POINT (-99.76349 32.42978)
97,"ABILENE, CITY OF (485450)",3309 PARKCREST DR,NaN,ABILENE,TX,79605,5024.0,"3309 PARKCREST DR, ABILENE, TX 79605","(3309, Parkcrest Drive, Abilene, Taylor County...",32.427916,-99.763816,None,POINT (-99.76382 32.42792)


In [21]:
#plot the geocoded points in leafmap
m = leafmap.Map(width=800, height=500)
address_gdf.explore(
    m=m,
    marker_type='marker',
    marker_kwds={
        'icon': folium.Icon(color='#fdbb84', icon='hurricane', prefix='fa')
    }
)
m.zoom_to_gdf(address_gdf)
m